In [ ]:
import sys
sys.path.insert(0, '/home/h/Opengate')
from config import scanner, phantom, reconstruction
import numpy as np

lor_data = np.load("/home/h/Opengate/Post_process/Outputs/coincidence_pairs.npz")
attenuation_map = np.load("/home/h/Opengate/Simulation/Outputs/mu_map.npy")
ground_truth_activity = np.load("/home/h/Opengate/Simulation/Outputs/activity_map.npy")

In [ ]:
"""
Analytical Sensitivity Map for Cylindrical PET Scanner
=======================================================
Implements ray-casting geometry from 2D cross-section, revolved for 3D.

For each point at (r, z):
- Near wall at distance (inner_radius - r)
- Far wall at distance (inner_radius + r)  
- Cast rays from far wall edges through point -> valid region on near wall
- Cast rays from near wall edges through point -> valid region on far wall
- Sensitivity = sum of valid detector lengths
"""

import numpy as np
from numba import njit, prange
import math


@njit
def compute_valid_length_on_wall(
    point_r: float,
    point_z: float,
    target_wall_dist: float,
    source_wall_dist: float,
    z_bottom: float,
    z_top: float
) -> float:
    """
    Compute valid detector length on target wall, limited by source wall edges.
    
    Uses similar triangles: a ray from source wall edge through the point
    projects onto the target wall at a z-offset scaled by the distance ratio.
    
    Args:
        point_r: Radial distance of point from axis (not used directly, 
                 but wall distances are derived from it)
        point_z: Axial position of point
        target_wall_dist: Distance from point to target wall
        source_wall_dist: Distance from point to source wall (opposite wall)
        z_bottom: Bottom of detector (axial)
        z_top: Top of detector (axial)
    
    Returns:
        Valid detector length on target wall (mm)
    """
    # Ray from source wall top edge through point -> where on target wall?
    # Similar triangles: z_offset scales with distance ratio
    # From source top (z_top) through point (point_z) to target wall
    # The ray continues past the point by ratio (target_wall_dist / source_wall_dist)
    
    total_dist = target_wall_dist + source_wall_dist
    
    if total_dist < 1e-10:
        return 0.0
    
    # Ray from source_top through point to target wall
    # Point is at fraction (source_wall_dist / total_dist) along the ray
    # z at target = point_z + (point_z - z_top) * (target_wall_dist / source_wall_dist)
    # Simplifies to: z_target = point_z - (z_top - point_z) * (target_wall_dist / source_wall_dist)
    
    ratio = target_wall_dist / source_wall_dist
    
    z_from_source_top = point_z - (z_top - point_z) * ratio
    z_from_source_bottom = point_z - (z_bottom - point_z) * ratio
    
    # The "shadow" region on target wall
    shadow_min = min(z_from_source_top, z_from_source_bottom)
    shadow_max = max(z_from_source_top, z_from_source_bottom)
    
    # Intersect with physical wall extent
    valid_min = max(z_bottom, shadow_min)
    valid_max = min(z_top, shadow_max)
    
    return max(0.0, valid_max - valid_min)


@njit
def compute_sensitivity_at_point(
    r: float,
    z: float,
    inner_radius: float,
    z_bottom: float,
    z_top: float
) -> float:
    """
    Compute sensitivity for a point at (r, z) in cylindrical coordinates.
    
    Sensitivity = valid_length_near_wall + valid_length_far_wall
    
    Args:
        r: Radial distance from central axis
        z: Axial position
        inner_radius: Scanner bore radius
        z_bottom: Bottom of detector
        z_top: Top of detector
    
    Returns:
        Sensitivity (total valid detector length in mm)
    """
    # Point must be inside bore
    if r >= inner_radius:
        return 0.0
    
    # Distance to near wall (closer side of cylinder)
    near_wall_dist = inner_radius - r
    # Distance to far wall (opposite side, through axis)
    far_wall_dist = inner_radius + r
    
    # Valid length on near wall (limited by far wall edges)
    near_length = compute_valid_length_on_wall(
        r, z, near_wall_dist, far_wall_dist, z_bottom, z_top
    )
    
    # Valid length on far wall (limited by near wall edges)
    far_length = compute_valid_length_on_wall(
        r, z, far_wall_dist, near_wall_dist, z_bottom, z_top
    )
    return near_length * far_length


@njit(parallel=True)
def generate_sensitivity_volume(
    n_voxels: int,
    voxel_size: float,
    half_fov: float,
    inner_radius: float,
    z_bottom: float,
    z_top: float
) -> np.ndarray:
    """
    Generate 3D sensitivity volume by revolving 2D sensitivity map.
    
    Args:
        n_voxels: Number of voxels per dimension
        voxel_size: Size of each voxel (mm)
        half_fov: Half of field of view (mm)
        inner_radius: Scanner bore radius (mm)
        z_bottom: Bottom of detector (mm)
        z_top: Top of detector (mm)
    
    Returns:
        3D sensitivity volume (n_voxels, n_voxels, n_voxels)
    """
    volume = np.zeros((n_voxels, n_voxels, n_voxels), dtype=np.float32)
    
    for i in prange(n_voxels):
        x = -half_fov + (i + 0.5) * voxel_size
        for j in range(n_voxels):
            y = -half_fov + (j + 0.5) * voxel_size
            r = math.sqrt(x * x + y * y)
            
            # Skip points outside bore
            if r >= inner_radius:
                continue
                
            for k in range(n_voxels):
                z = -half_fov + (k + 0.5) * voxel_size
                volume[i, j, k] = compute_sensitivity_at_point(
                    r, z, inner_radius, z_bottom, z_top
                )
    
    return volume


def create_sensitivity_map(scanner, reconstruction):
    """
    Create sensitivity map from scanner and reconstruction config.
    
    Args:
        scanner: Scanner config object with inner_radius, z_bottom, z_top
        reconstruction: Reconstruction config with voxel_size, fov, n_voxels
    
    Returns:
        3D numpy array of sensitivity values
    """
    n_voxels = reconstruction.n_voxels
    voxel_size = reconstruction.voxel_size
    half_fov = reconstruction.half_fov
    
    print(f"Generating sensitivity map...")
    print(f"  Scanner: inner_radius={scanner.inner_radius:.1f}mm, "
          f"axial_length={scanner.axial_length:.1f}mm")
    print(f"  Image: {n_voxels}³ voxels ({voxel_size}mm isotropic)")
    
    sensitivity = generate_sensitivity_volume(
        n_voxels=n_voxels,
        voxel_size=voxel_size,
        half_fov=half_fov,
        inner_radius=scanner.inner_radius,
        z_bottom=scanner.z_bottom,
        z_top=scanner.z_top
    )
    
    # Avoid division by zero in reconstruction
    sensitivity = np.maximum(sensitivity, 1e-10)
    
    print(f"  Sensitivity range: [{sensitivity.min():.2f}, {sensitivity.max():.2f}] mm")
    print(f"  Non-zero voxels: {np.sum(sensitivity > 1e-10):,}")
    
    return sensitivity


sensitivity = create_sensitivity_map(scanner, reconstruction)

# Quick validation
n = reconstruction.n_voxels
centre = n // 2

print(f"\nValidation:")
print(f"  Centre sensitivity: {sensitivity[centre, centre, centre]:.2f} mm")
print(f"  Max sensitivity: {sensitivity.max():.2f} mm")

# Check symmetry
lr_diff = np.abs(sensitivity - np.flip(sensitivity, axis=0)).max()
tb_diff = np.abs(sensitivity - np.flip(sensitivity, axis=1)).max()
ax_diff = np.abs(sensitivity - np.flip(sensitivity, axis=2)).max()
print(f"  Symmetry errors (should be ~0): LR={lr_diff:.6f}, TB={tb_diff:.6f}, Ax={ax_diff:.6f}")

In [ ]:
from numba import njit, prange
import math

@njit
def compute_sensitivity_2d(r, z, inner_radius, z_bottom, z_top, axial_length):
    if r >= inner_radius:
        return 0.0
    
    d_to_wall = inner_radius - r
    
    z_min_near = z - d_to_wall * (z - z_bottom) / inner_radius
    z_max_near = z + d_to_wall * (z_top - z) / inner_radius
    z_min_near = max(z_bottom, z_min_near)
    z_max_near = min(z_top, z_max_near)
    
    d_through = inner_radius + r
    scale = d_through / (inner_radius + r) if (inner_radius + r) > 0 else 1
    z_min_far = z - (z - z_bottom) * scale
    z_max_far = z + (z_top - z) * scale
    z_min_far = max(z_bottom, z_min_far)
    z_max_far = min(z_top, z_max_far)
    
    range_near = max(0.0, z_max_near - z_min_near)
    range_far = max(0.0, z_max_far - z_min_far)
    
    sensitivity = range_near * range_far / (axial_length ** 2)
    return sensitivity


@njit(parallel=True)
def generate_sensitivity_volume(n_voxels, voxel_size, fov, inner_radius, z_bottom, z_top, axial_length):
    half_fov = fov / 2
    volume = np.zeros((n_voxels, n_voxels, n_voxels), dtype=np.float32)
    
    for i in prange(n_voxels):
        x = -half_fov + (i + 0.5) * voxel_size
        for j in range(n_voxels):
            y = -half_fov + (j + 0.5) * voxel_size
            r = math.sqrt(x**2 + y**2)
            
            for k in range(n_voxels):
                z = -half_fov + (k + 0.5) * voxel_size
                volume[i, j, k] = compute_sensitivity_2d(r, z, inner_radius, z_bottom, z_top, axial_length)
    
    return volume


def create_sensitivity_map(voxel_size, fov):
    """
    Wrapper to create sensitivity map with scanner geometry.
    """
    n_voxels = int(fov / voxel_size)
    
    print(f"Generating sensitivity map...")
    print(f"  Scanner: inner_radius={scanner.inner_radius:.1f}mm, axial_length={scanner.axial_length:.1f}mm")
    print(f"  Image: {n_voxels}³ voxels ({voxel_size}mm)")
    
    sensitivity = generate_sensitivity_volume(
        n_voxels, voxel_size, fov,
        scanner.inner_radius, scanner.z_bottom,
        scanner.z_top, scanner.axial_length
    )
    
    # Avoid division by zero - set minimum sensitivity
    sensitivity = np.maximum(sensitivity, 1e-10)
    
    print(f"  Range: [{sensitivity.min():.6f}, {sensitivity.max():.6f}]")
    print(f"  Non-zero voxels: {np.sum(sensitivity > 1e-10):,}")
    
    return sensitivity


sensitivity = create_sensitivity_map(voxel_size=reconstruction.voxel_size, fov=reconstruction.fov)

In [ ]:
import plotly.express as px

# sensitivity = sensitivity * 0 + 0.0001
# Animate through slices
fig = px.imshow(
    sensitivity,
    animation_frame=2,
    color_continuous_scale='Hot',
).show()

In [ ]:
# =============================================================================
# SIDDON PROJECTOR WITH ATTENUATION
# =============================================================================
@njit
def siddon_single_lor_with_attenuation(p1, p2, image, mu_map, n_voxels, voxel_size, half_fov, forward=True):
    """
    Siddon's algorithm with attenuation correction.
    
    For forward projection: returns attenuated line integral
    For back projection: returns attenuation-weighted contribution to each voxel
    
    Args:
        p1, p2: LOR endpoints
        image: Activity image (for forward) or will be modified (for back)
        mu_map: Attenuation map (μ in mm^-1)
        forward: If True, compute forward projection. If False, backproject.
    
    Returns:
        For forward: line integral value
        For back: None (modifies image in place)
    """
    # Convert to voxel coordinates
    p1_v = np.empty(3)
    p2_v = np.empty(3)
    for i in range(3):
        p1_v[i] = (p1[i] + half_fov) / voxel_size
        p2_v[i] = (p2[i] + half_fov) / voxel_size
    
    # Direction vector
    d = np.empty(3)
    for i in range(3):
        d[i] = p2_v[i] - p1_v[i]
    
    lor_length = np.sqrt(d[0]**2 + d[1]**2 + d[2]**2)
    
    if lor_length < 1e-10:
        return 0.0
    
    # Compute alpha bounds
    alpha_min = 0.0
    alpha_max = 1.0
    
    for i in range(3):
        if abs(d[i]) > 1e-10:
            a1 = (0 - p1_v[i]) / d[i]
            a2 = (n_voxels - p1_v[i]) / d[i]
            if a1 > a2:
                a1, a2 = a2, a1
            alpha_min = max(alpha_min, a1)
            alpha_max = min(alpha_max, a2)
    
    if alpha_min >= alpha_max:
        return 0.0
    
    # Collect alpha values at voxel boundaries
    max_intersections = 3 * n_voxels + 3
    alphas = np.empty(max_intersections, dtype=np.float64)
    n_alphas = 0
    
    alphas[n_alphas] = alpha_min
    n_alphas += 1
    alphas[n_alphas] = alpha_max
    n_alphas += 1
    
    for axis in range(3):
        if abs(d[axis]) < 1e-10:
            continue
        
        if d[axis] > 0:
            i_min = int(np.ceil(p1_v[axis] + alpha_min * d[axis]))
            i_max = int(np.floor(p1_v[axis] + alpha_max * d[axis]))
        else:
            i_min = int(np.ceil(p1_v[axis] + alpha_max * d[axis]))
            i_max = int(np.floor(p1_v[axis] + alpha_min * d[axis]))
        
        i_min = max(0, min(n_voxels, i_min))
        i_max = max(0, min(n_voxels, i_max))
        
        for i in range(i_min, i_max + 1):
            alpha = (i - p1_v[axis]) / d[axis]
            if alpha_min < alpha < alpha_max:
                alphas[n_alphas] = alpha
                n_alphas += 1
    
    # Sort alphas
    alphas_sorted = np.sort(alphas[:n_alphas])
    
    # Remove duplicates
    unique_alphas = np.empty(n_alphas, dtype=np.float64)
    n_unique = 0
    prev = -1e10
    for i in range(len(alphas_sorted)):
        if alphas_sorted[i] - prev > 1e-10:
            unique_alphas[n_unique] = alphas_sorted[i]
            n_unique += 1
            prev = alphas_sorted[i]
    
    if n_unique < 2:
        return 0.0
    
    # First pass: compute total attenuation along LOR (for ACF)
    total_mu_length = 0.0
    for i in range(n_unique - 1):
        alpha_mid = (unique_alphas[i] + unique_alphas[i + 1]) / 2.0
        
        ix = int(p1_v[0] + alpha_mid * d[0])
        iy = int(p1_v[1] + alpha_mid * d[1])
        iz = int(p1_v[2] + alpha_mid * d[2])
        
        if 0 <= ix < n_voxels and 0 <= iy < n_voxels and 0 <= iz < n_voxels:
            segment_length = (unique_alphas[i + 1] - unique_alphas[i]) * lor_length * voxel_size
            total_mu_length += mu_map[ix, iy, iz] * segment_length
    
    # Attenuation correction factor
    acf = np.exp(total_mu_length) if total_mu_length > 0 else 1.0
    
    if forward:
        # Forward projection: compute line integral with attenuation
        line_integral = 0.0
        
        for i in range(n_unique - 1):
            alpha_mid = (unique_alphas[i] + unique_alphas[i + 1]) / 2.0
            
            ix = int(p1_v[0] + alpha_mid * d[0])
            iy = int(p1_v[1] + alpha_mid * d[1])
            iz = int(p1_v[2] + alpha_mid * d[2])
            
            if 0 <= ix < n_voxels and 0 <= iy < n_voxels and 0 <= iz < n_voxels:
                segment_length = (unique_alphas[i + 1] - unique_alphas[i]) * lor_length * voxel_size
                line_integral += image[ix, iy, iz] * segment_length
        
        # Apply attenuation (photons are attenuated, so divide by ACF)
        return line_integral / acf
    
    else:
        # Backprojection: distribute value to voxels with attenuation weighting
        return acf  # Return ACF for external handling


@njit
def forward_project_lor(p1, p2, image, mu_map, n_voxels, voxel_size, half_fov):
    """Forward project a single LOR with attenuation."""
    return siddon_single_lor_with_attenuation(p1, p2, image, mu_map, n_voxels, voxel_size, half_fov, forward=True)


@njit
def siddon_backproject_lor(p1, p2, n_voxels, voxel_size, half_fov, value, image):
    """
    Backproject a single LOR value into image.
    """
    # Convert to voxel coordinates
    p1_v = np.empty(3)
    p2_v = np.empty(3)
    for i in range(3):
        p1_v[i] = (p1[i] + half_fov) / voxel_size
        p2_v[i] = (p2[i] + half_fov) / voxel_size
    
    d = np.empty(3)
    for i in range(3):
        d[i] = p2_v[i] - p1_v[i]
    
    lor_length = np.sqrt(d[0]**2 + d[1]**2 + d[2]**2)
    
    if lor_length < 1e-10:
        return
    
    alpha_min = 0.0
    alpha_max = 1.0
    
    for i in range(3):
        if abs(d[i]) > 1e-10:
            a1 = (0 - p1_v[i]) / d[i]
            a2 = (n_voxels - p1_v[i]) / d[i]
            if a1 > a2:
                a1, a2 = a2, a1
            alpha_min = max(alpha_min, a1)
            alpha_max = min(alpha_max, a2)
    
    if alpha_min >= alpha_max:
        return
    
    max_intersections = 3 * n_voxels + 3
    alphas = np.empty(max_intersections, dtype=np.float64)
    n_alphas = 0
    
    alphas[n_alphas] = alpha_min
    n_alphas += 1
    alphas[n_alphas] = alpha_max
    n_alphas += 1
    
    for axis in range(3):
        if abs(d[axis]) < 1e-10:
            continue
        
        if d[axis] > 0:
            i_min = int(np.ceil(p1_v[axis] + alpha_min * d[axis]))
            i_max = int(np.floor(p1_v[axis] + alpha_max * d[axis]))
        else:
            i_min = int(np.ceil(p1_v[axis] + alpha_max * d[axis]))
            i_max = int(np.floor(p1_v[axis] + alpha_min * d[axis]))
        
        i_min = max(0, min(n_voxels, i_min))
        i_max = max(0, min(n_voxels, i_max))
        
        for i in range(i_min, i_max + 1):
            alpha = (i - p1_v[axis]) / d[axis]
            if alpha_min < alpha < alpha_max:
                alphas[n_alphas] = alpha
                n_alphas += 1
    
    alphas_sorted = np.sort(alphas[:n_alphas])
    
    unique_alphas = np.empty(n_alphas, dtype=np.float64)
    n_unique = 0
    prev = -1e10
    for i in range(len(alphas_sorted)):
        if alphas_sorted[i] - prev > 1e-10:
            unique_alphas[n_unique] = alphas_sorted[i]
            n_unique += 1
            prev = alphas_sorted[i]
    
    if n_unique < 2:
        return
    
    for i in range(n_unique - 1):
        alpha_mid = (unique_alphas[i] + unique_alphas[i + 1]) / 2.0
        
        ix = int(p1_v[0] + alpha_mid * d[0])
        iy = int(p1_v[1] + alpha_mid * d[1])
        iz = int(p1_v[2] + alpha_mid * d[2])
        
        if 0 <= ix < n_voxels and 0 <= iy < n_voxels and 0 <= iz < n_voxels:
            segment_length = (unique_alphas[i + 1] - unique_alphas[i]) * lor_length * voxel_size
            image[ix, iy, iz] += value * segment_length


# =============================================================================
# OSEM RECONSTRUCTION
# =============================================================================
def osem_reconstruct(xyz1, xyz2, mu_map=None, voxel_size=1.0, fov=200.0, 
                     n_iterations=2, n_subsets=8, verbose=True):
    """
    OSEM reconstruction with attenuation correction.
    
    Args:
        xyz1, xyz2: LOR endpoints (N x 3 arrays)
        mu_map: Attenuation map array (if None, generates analytically)
        voxel_size: Voxel size in mm
        fov: Field of view in mm
        n_iterations: Number of full iterations
        n_subsets: Number of subsets per iteration
        verbose: Print progress
    
    Returns:
        Reconstructed image (3D array)
    """
    n_voxels = int(fov / voxel_size)
    half_fov = fov / 2
    n_lors = len(xyz1)
    
    if verbose:
        print(f"OSEM Reconstruction")
        print(f"  LORs: {n_lors:,}")
        print(f"  Image: {n_voxels}³ voxels ({voxel_size}mm)")
        print(f"  Iterations: {n_iterations}, Subsets: {n_subsets}")
    
    # Load or generate attenuation map
    if verbose:
        print("\nAttenuation map...")
    
    if verbose:
        print(f"  Using provided μ-map, shape: {mu_map.shape}")
    
    if mu_map.shape != (n_voxels, n_voxels, n_voxels):
        raise ValueError(
            f"μ-map shape {mu_map.shape} doesn't match expected "
            f"({n_voxels}, {n_voxels}, {n_voxels}). "
            f"Check voxel_size and fov parameters."
        )
    
    mu_map = mu_map.astype(np.float32)

    if verbose:
        print(f"  Non-zero voxels: {np.sum(mu_map > 0):,}")
        print(f"  μ range: [{mu_map.min():.6f}, {mu_map.max():.6f}] mm⁻¹")
    # Initialise image (uniform positive value)
    image = np.ones((n_voxels, n_voxels, n_voxels), dtype=np.float32)
    
    # Create subset indices
    subset_indices = [np.arange(i, n_lors, n_subsets) for i in range(n_subsets)]
    
    if verbose:
        print(f"\nStarting iterations...")
    
    # OSEM iterations
    for iteration in range(n_iterations):
        for subset_idx in range(n_subsets):
            indices = subset_indices[subset_idx]
            n_subset_lors = len(indices)
            
            # Forward project current estimate
            forward_proj = np.zeros(n_subset_lors, dtype=np.float32)
            
            for i, lor_idx in enumerate(indices):
                p1 = xyz1[lor_idx].astype(np.float64)
                p2 = xyz2[lor_idx].astype(np.float64)
                forward_proj[i] = forward_project_lor(
                    p1, p2, image, mu_map, n_voxels, voxel_size, half_fov
                )
            
            # Compute ratio (measured / estimated)
            # Measured = 1 for each detected LOR
            ratios = np.ones(n_subset_lors, dtype=np.float32)
            mask = forward_proj > 1e-10
            ratios[mask] = 1.0 / forward_proj[mask]
            
            # Backproject ratios
            backproj = np.zeros((n_voxels, n_voxels, n_voxels), dtype=np.float32)
            
            for i, lor_idx in enumerate(indices):
                p1 = xyz1[lor_idx].astype(np.float64)
                p2 = xyz2[lor_idx].astype(np.float64)
                siddon_backproject_lor(
                    p1, p2, n_voxels, voxel_size, half_fov, ratios[i], backproj
                )
            
            # Update image: image *= backproj / sensitivity
            # Scale by number of subsets to account for subset sampling
            update = backproj / sensitivity
            image *= update
            
            # Enforce positivity
            image = np.maximum(image, 1e-10)
            
            if verbose:
                print(f"  Iteration {iteration+1}/{n_iterations}, "
                      f"Subset {subset_idx+1}/{n_subsets}, "
                      f"Image range: [{image.min():.4f}, {image.max():.4f}]")
    
    if verbose:
        print("\nReconstruction complete!")
    
    return image


# =============================================================================
# RUN RECONSTRUCTION
# =============================================================================
# Assuming 'data' contains xyz1 and xyz2 from your LOR extraction
osem_image = osem_reconstruct(
    lor_data['xyz1'], lor_data['xyz2'],
    mu_map=attenuation_map,
    voxel_size=reconstruction.voxel_size,
    fov=reconstruction.fov,
    n_iterations=1,
    n_subsets=4,
)

# np.save("osem_reconstruction.npy", osem_image) # Save result

In [ ]:
import plotly.express as px

# Animate through slices
fig = px.imshow(
    osem_image,
    animation_frame=2,
    zmax=osem_image.max(),
    color_continuous_scale='Hot',
).show()

fig = px.imshow(
    ground_truth_activity,
    animation_frame=2,
    zmax=ground_truth_activity.max(),
    color_continuous_scale='Hot',
).show()